# Subscriber Cancellations Data pipeline

## setup and helper functions

In [1]:
import pandas as pd
import sys
import json
import sqlite3
from IPython.display import display
import logging

In [2]:
#function to establish connection and create dataframes.
def load_data():
    try:
        conn = sqlite3.connect("cademycode.db")
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        table_names = [row[0] for row in cursor.fetchall()]

        dfs = {}
        for table in table_names:
            dfs[table] = pd.read_sql_query(f"SELECT * FROM {table}", conn)

        df_students = dfs["cademycode_students"]
        df_courses = dfs["cademycode_courses"]
        df_jobs = dfs["cademycode_student_jobs"]

        return df_students, df_courses, df_jobs

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None, None

    finally:
        if conn:
            conn.close()


In [3]:
def display_overview(df):

    # Display basic info about columns and data types
    print("Column Names and Data Types:")
    print(df.dtypes)
    print("\n" + "-"*50 + "\n")

    # Display the first few rows of the DataFrame
    print("First 5 Rows:")
    print(df.head())
    print("\n" + "-"*50 + "\n")
    
    # Display summary statistics for numeric columns
    print("Summary Statistics (Describe):")
    print(df.describe())
    print("\n" + "-"*50 + "\n")
    
    # Display number of missing values per column
    print("Missing Values Count:")
    print(df.isnull().sum())
    print("\n" + "-"*50 + "\n")

In [4]:
# function to split json data into columns. The parameter new_columns is a list of column names.
def split_json_column(df, json_column, new_columns):
    # Check if the column exists
    if json_column not in df.columns:
        raise ValueError(f"Column '{json_column}' not found in the DataFrame.")
    
    # Split the JSON data in the specified column and create new columns
    try:
        df[new_columns] = df[json_column].apply(lambda x: pd.Series(json.loads(x) if isinstance(x, str) else x))
    except Exception as e:
        raise ValueError(f"Error parsing JSON in column '{json_column}': {e}")
    
    # Drop the original JSON column after splitting
    df.drop(columns=[json_column], inplace=True)
    
    return df

In [5]:
# function to fill missing data. The filler must be the same type of column type.
def process_missing_data(df, columns_to_fill, subset_for_dropna=None, filler='0'):
    if subset_for_dropna:
        df = df.dropna(subset=subset_for_dropna)
    
    for column in columns_to_fill:
        df.loc[:, column] = df[column].fillna(filler)
    
    return df

In [6]:
def merge_dataframes(df_students, df_jobs, df_courses):
    try:
        # Convert job_id and current_career_path_id to int64
        df_students["job_id"] = pd.to_numeric(df_students["job_id"], errors='coerce').astype("Int64")  # Using 'Int64' to allow NaN
        df_students["current_career_path_id"] = pd.to_numeric(df_students["current_career_path_id"], errors='coerce').astype("Int64")
        
        # Merge with df_jobs and df_courses
        merged_df = pd.merge(df_students, df_jobs, on="job_id", how="left")
        merged_df = pd.merge(merged_df, df_courses, left_on="current_career_path_id", right_on="career_path_id", how="left")
        
        # Fill missing values and convert column types
        merged_df['career_path_id'] = merged_df['career_path_id'].fillna(0).astype('int64')
        merged_df['career_path_name'] = merged_df['career_path_name'].fillna('No Career Path')
        
        # Check for duplicates
        duplicates = merged_df.duplicated(keep=False)
        
        return merged_df, duplicates
    
    except KeyError as e:
        print(f"KeyError occurred while merging: {e}")
    except ValueError as e:
        print(f"ValueError occurred during type conversion: {e}")
    except Exception as e:
        print(f"An unexpected error occurred when merging: {e}")
        return None, None

In [7]:
#function to export cleaned data to CSV file

def export_to_sqlite_and_csv(df, db_name='clean_data.db', table_name='students', csv_filename='final_output.csv'):
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_name)

        # Save DataFrame to the database
        df.to_sql(table_name, conn, if_exists='replace', index=False)

        # Confirm the table exists
        tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
        print("Tables in database:", tables['name'].tolist())

        # Fetch all rows from the table to a new DataFrame
        result_df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

        # Export to CSV
        result_df.to_csv(csv_filename, index=False)
        print(f"CSV file '{csv_filename}' has been created successfully!")

    except sqlite3.Error as e:
        print(f"SQLite error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")


## General Overview

In [8]:
df_students, df_courses, df_jobs = load_data()
display_overview(df_students)
display_overview(df_courses)
display_overview(df_jobs)


Column Names and Data Types:
uuid                       int64
name                      object
dob                       object
sex                       object
contact_info              object
job_id                    object
num_course_taken          object
current_career_path_id    object
time_spent_hrs            object
dtype: object

--------------------------------------------------

First 5 Rows:
   uuid             name         dob sex  \
0     1  Annabelle Avery  1943-07-03   F   
1     2      Micah Rubio  1991-02-07   M   
2     3       Hosea Dale  1989-12-07   M   
3     4     Mariann Kirk  1988-07-31   F   
4     5  Lucio Alexander  1963-08-31   M   

                                        contact_info job_id num_course_taken  \
0  {"mailing_address": "303 N Timber Key, Irondal...    7.0              6.0   
1  {"mailing_address": "767 Crescent Fair, Shoals...    7.0              5.0   
2  {"mailing_address": "P.O. Box 41269, St. Bonav...    7.0              8.0   
3  {"mai

## df_students Cleaning

In [9]:
# Split the column contact_info into mailing_address and email
split_json_column(df_students, 'contact_info', ['mailing_address', 'email'])
display(df_students[['mailing_address', 'email']].head(3))
print(df_students.columns)


,mailing_address,email
0,"303 N Timber Key, Irondale, Wisconsin, 84736",annabelle_avery9376@woohoo.com
1,"767 Crescent Fair, Shoals, Indiana, 37439",rubio6772@hmail.com
2,"P.O. Box 41269, St. Bonaventure, Virginia, 83637",hosea_dale8084@coldmail.com


Index(['uuid', 'name', 'dob', 'sex', 'job_id', 'num_course_taken',
       'current_career_path_id', 'time_spent_hrs', 'mailing_address', 'email'],
      dtype='object')


In [10]:
# dealing with missing values


# 5 at job_id
# 251 at num_course_taken
# 471 at current_career_path_id and time_spent_hrs. When student does not have selected career path, the time_spent_hrs is automatically missing.
# Since only 5 missing data at job_id, we are dropping them
# Filling missing data with 0s, since missing data represent no course taken so far. It's the same for career path; missing data represents no career path taken and no time spent on this career path.
df_students = process_missing_data(
    df_students,
    columns_to_fill=['num_course_taken', 'current_career_path_id', 'time_spent_hrs'],
    subset_for_dropna=['job_id'],
    filler='0'
)

In [11]:

print("Missing Values Count:")
print(df_students.isnull().sum())
print("\n" + "-"*50 + "\n")


Missing Values Count:
uuid                      0
name                      0
dob                       0
sex                       0
job_id                    0
num_course_taken          0
current_career_path_id    0
time_spent_hrs            0
mailing_address           0
email                     0
dtype: int64

--------------------------------------------------



In [12]:
# Dealing with data types

df_students['dob'] = pd.to_datetime(df_students['dob'])
df_students['num_course_taken'] = pd.to_numeric(df_students['num_course_taken'], errors='coerce')
df_students['current_career_path_id'] = pd.to_numeric(df_students['current_career_path_id'], errors='coerce')
df_students['job_id'] = pd.to_numeric(df_students['job_id'], errors='coerce')
df_students['num_course_taken'] = df_students['num_course_taken'].astype('Int64')
df_students['current_career_path_id'] = df_students['current_career_path_id'].astype('int64')
df_students['time_spent_hrs'] = df_students['time_spent_hrs'].astype('float64')
df_students['job_id'] = df_students['job_id'].astype('int64')
print(df_students.dtypes)

uuid                               int64
name                              object
dob                       datetime64[ns]
sex                               object
job_id                             int64
num_course_taken                   Int64
current_career_path_id             int64
time_spent_hrs                   float64
mailing_address                   object
email                             object
dtype: object


In [13]:
# Dropping duplicates
df_students = df_students.drop_duplicates(subset=['uuid'], keep='first')
df_students.shape

(4995, 10)

## df_jobs cleaning

In [14]:
# Display the entire jobs DataFrame
print("Jobs DataFrame:")
print(df_jobs)

# Display the entire courses DataFrame
print("\nCourses DataFrame:")
print(df_courses)

df_jobs = df_jobs.drop_duplicates(keep='first')


Jobs DataFrame:
    job_id        job_category  avg_salary
0        1           analytics       86000
1        2            engineer      101000
2        3  software developer      110000
3        4            creative       66000
4        5  financial services      135000
5        6           education       61000
6        7                  HR       80000
7        8             student       10000
8        9          healthcare      120000
9        0               other       80000
10       3  software developer      110000
11       4            creative       66000
12       5  financial services      135000

Courses DataFrame:
   career_path_id           career_path_name  hours_to_complete
0               1             data scientist                 20
1               2              data engineer                 20
2               3               data analyst                 12
3               4       software engineering                 25
4               5           backend engine

In [15]:
df_jobs = df_jobs.drop_duplicates(keep='first')
df_jobs.shape

(10, 3)

In [16]:
# No modifications required for the courses dataframe

## Merge

In [17]:
final_df, duplicates = merge_dataframes(df_students, df_jobs, df_courses)

print(f"Number of duplicate rows: {duplicates.sum()}")
print(final_df[duplicates])
print(display(final_df.shape))
display(final_df.head())
#filling hours_to_complet with 0s since there is no career path taken in all those cases.
final_df = process_missing_data(final_df, ['hours_to_complete'], subset_for_dropna=None, filler=0)
print(display_overview(final_df))

Number of duplicate rows: 0
Empty DataFrame
Columns: [uuid, name, dob, sex, job_id, num_course_taken, current_career_path_id, time_spent_hrs, mailing_address, email, job_category, avg_salary, career_path_id, career_path_name, hours_to_complete]
Index: []


(4995, 15)

None


,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,mailing_address,email,job_category,avg_salary,career_path_id,career_path_name,hours_to_complete
0,1,Annabelle Avery,1943-07-03,F,7,6,1,4.99,"303 N Timber Key, Irondale, Wisconsin, 84736",annabelle_avery9376@woohoo.com,HR,80000,1,data scientist,20.0
1,2,Micah Rubio,1991-02-07,M,7,5,8,4.40,"767 Crescent Fair, Shoals, Indiana, 37439",rubio6772@hmail.com,HR,80000,8,android developer,27.0
2,3,Hosea Dale,1989-12-07,M,7,8,8,6.74,"P.O. Box 41269, St. Bonaventure, Virginia, 83637",hosea_dale8084@coldmail.com,HR,80000,8,android developer,27.0
3,4,Mariann Kirk,1988-07-31,F,6,7,9,12.31,"517 SE Wintergreen Isle, Lane, Arkansas, 82242",kirk4005@hmail.com,education,61000,9,machine learning engineer,35.0
4,5,Lucio Alexander,1963-08-31,M,7,14,3,5.64,"18 Cinder Cliff, Doyles borough, Rhode Island,...",alexander9810@hmail.com,HR,80000,3,data analyst,12.0


Column Names and Data Types:
uuid                               int64
name                              object
dob                       datetime64[ns]
sex                               object
job_id                             Int64
num_course_taken                   Int64
current_career_path_id             Int64
time_spent_hrs                   float64
mailing_address                   object
email                             object
job_category                      object
avg_salary                         int64
career_path_id                     int64
career_path_name                  object
hours_to_complete                float64
dtype: object

--------------------------------------------------

First 5 Rows:
   uuid             name        dob sex  job_id  num_course_taken  \
0     1  Annabelle Avery 1943-07-03   F       7                 6   
1     2      Micah Rubio 1991-02-07   M       7                 5   
2     3       Hosea Dale 1989-12-07   M       7                 8   

## Export to CSV

In [18]:
export_to_sqlite_and_csv(final_df, db_name='clean_data.db', table_name='students', csv_filename='final_output.csv')

Tables in database: ['students']
CSV file 'final_output.csv' has been created successfully!
Database connection closed.
